In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sn
from xgboost.sklearn import XGBClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, MaxPooling1D, Conv1D, AlphaDropout


import warnings; warnings.simplefilter('ignore')
%matplotlib inline

/home/silveira/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

In [3]:
dataset = pd.read_csv('epileptic_seizure_recognition.csv')

In [4]:
df = dataset[dataset.y != 4]
df = df[df.y != 3]

In [8]:
tgt = df.y
tgt = tgt.replace([1,2,5],[0,1,2])

X = df.iloc[:,1:179].values
y = tgt.iloc[:].values

import scipy.sparse
indptr = range(len(y)+1)
data = np.ones(len(y))
y = scipy.sparse.csr_matrix((data, y, indptr))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_acc',
                                           min_delta=0,
                                           patience=10000,
                                           verbose=1,
                                           mode='auto')
from keras.callbacks import ModelCheckpoint
filepath="weights_ZvsFvsS_sgd_simples.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Initialising the ANN
clf_ann = Sequential()

# Adding the input layer and the first hidden layer
clf_ann.add(Dense(activation="relu", 
                  units=178,
                  input_dim=178,
                  kernel_initializer='he_normal'))
clf_ann.add(BatchNormalization())
clf_ann.add(Dropout(0.5))
# Adding the second hidden layer
clf_ann.add(Dense(activation="relu",
                  units=178,
                  kernel_initializer='he_normal'))
clf_ann.add(BatchNormalization())
clf_ann.add(Dropout(0.5))
# Adding the output layer
clf_ann.add(Dense(units=3,
                  activation="softmax"))

# Compiling the ANN
clf_ann.compile(optimizer=keras.optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
# loss = 'categorical_crossentropy'
# Fitting the ANN to the Training set
clf_ann.fit(X_train, y_train,
               batch_size = 256,
               epochs = 300,
               validation_data=(X_test,y_test),
               callbacks=[early_stop, checkpoint])

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = clf_ann.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
#cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred, target_names=['Seizure', 'Tumor Area','Healthy']))

#df_cm = pd.DataFrame(cm, range(3), range(3))
#sn.heatmap(df_cm, annot=True,fmt='g',cmap ='Blues')# font size

Train on 5520 samples, validate on 1380 samples
Epoch 1/300
5520/5520 [==============================] - 2s 309us/step - loss: 0.8717 - acc: 0.6123 - val_loss: 0.6469 - val_acc: 0.6913

Epoch 00001: val_acc improved from -inf to 0.69130, saving model to weights_ZvsFvsS_sgd_simples.best.h5
Epoch 2/300
5520/5520 [==============================] - 1s 117us/step - loss: 0.6424 - acc: 0.6732 - val_loss: 0.5543 - val_acc: 0.7297

Epoch 00002: val_acc improved from 0.69130 to 0.72971, saving model to weights_ZvsFvsS_sgd_simples.best.h5
Epoch 3/300
5520/5520 [==============================] - 0s 89us/step - loss: 0.5926 - acc: 0.6855 - val_loss: 0.5248 - val_acc: 0.7239

Epoch 00003: val_acc did not improve
Epoch 4/300
5520/5520 [==============================] - 1s 118us/step - loss: 0.5589 - acc: 0.6947 - val_loss: 0.5062 - val_acc: 0.7268

Epoch 00004: val_acc did not improve
Epoch 5/300
5520/5520 [==============================] - 1s 111us/step - loss: 0.5492 - acc: 0.7014 - val_loss: 0.48

5520/5520 [==============================] - 1s 107us/step - loss: 0.2532 - acc: 0.8987 - val_loss: 0.2297 - val_acc: 0.9123

Epoch 00041: val_acc improved from 0.90942 to 0.91232, saving model to weights_ZvsFvsS_sgd_simples.best.h5
Epoch 42/300
5520/5520 [==============================] - 1s 106us/step - loss: 0.2435 - acc: 0.8980 - val_loss: 0.3520 - val_acc: 0.8326

Epoch 00042: val_acc did not improve
Epoch 43/300
5520/5520 [==============================] - 1s 109us/step - loss: 0.2382 - acc: 0.9031 - val_loss: 0.3160 - val_acc: 0.8580

Epoch 00043: val_acc did not improve
Epoch 44/300
5520/5520 [==============================] - 1s 101us/step - loss: 0.2461 - acc: 0.9024 - val_loss: 0.2491 - val_acc: 0.8790

Epoch 00044: val_acc did not improve
Epoch 45/300
5520/5520 [==============================] - 1s 104us/step - loss: 0.2182 - acc: 0.9145 - val_loss: 0.3288 - val_acc: 0.8406

Epoch 00045: val_acc did not improve
Epoch 46/300
5520/5520 [==============================] - 1s 12


Epoch 00085: val_acc did not improve
Epoch 86/300
5520/5520 [==============================] - 1s 106us/step - loss: 0.1835 - acc: 0.9261 - val_loss: 0.2004 - val_acc: 0.9181

Epoch 00086: val_acc did not improve
Epoch 87/300
5520/5520 [==============================] - 1s 106us/step - loss: 0.1724 - acc: 0.9335 - val_loss: 0.2391 - val_acc: 0.9072

Epoch 00087: val_acc did not improve
Epoch 88/300
5520/5520 [==============================] - 1s 100us/step - loss: 0.1653 - acc: 0.9342 - val_loss: 0.2061 - val_acc: 0.9145

Epoch 00088: val_acc did not improve
Epoch 89/300
5520/5520 [==============================] - 1s 97us/step - loss: 0.1847 - acc: 0.9297 - val_loss: 0.2687 - val_acc: 0.8848

Epoch 00089: val_acc did not improve
Epoch 90/300
5520/5520 [==============================] - 1s 97us/step - loss: 0.1630 - acc: 0.9313 - val_loss: 0.2361 - val_acc: 0.9022

Epoch 00090: val_acc did not improve
Epoch 91/300
5520/5520 [==============================] - 1s 125us/step - loss: 0.16


Epoch 00130: val_acc did not improve
Epoch 131/300
5520/5520 [==============================] - 1s 114us/step - loss: 0.1471 - acc: 0.9424 - val_loss: 0.2400 - val_acc: 0.9022

Epoch 00131: val_acc did not improve
Epoch 132/300
5520/5520 [==============================] - 1s 120us/step - loss: 0.1409 - acc: 0.9453 - val_loss: 0.2235 - val_acc: 0.9159

Epoch 00132: val_acc did not improve
Epoch 133/300
5520/5520 [==============================] - 1s 115us/step - loss: 0.1393 - acc: 0.9444 - val_loss: 0.1972 - val_acc: 0.9217

Epoch 00133: val_acc did not improve
Epoch 134/300
5520/5520 [==============================] - 1s 115us/step - loss: 0.1492 - acc: 0.9438 - val_loss: 0.2439 - val_acc: 0.9000

Epoch 00134: val_acc did not improve
Epoch 135/300
5520/5520 [==============================] - 1s 114us/step - loss: 0.1343 - acc: 0.9480 - val_loss: 0.2101 - val_acc: 0.9145

Epoch 00135: val_acc did not improve
Epoch 136/300
5520/5520 [==============================] - 1s 111us/step - lo

5520/5520 [==============================] - 1s 95us/step - loss: 0.1076 - acc: 0.9591 - val_loss: 0.1964 - val_acc: 0.9225

Epoch 00175: val_acc did not improve
Epoch 176/300
5520/5520 [==============================] - 1s 96us/step - loss: 0.1264 - acc: 0.9520 - val_loss: 0.2046 - val_acc: 0.9217

Epoch 00176: val_acc did not improve
Epoch 177/300
5520/5520 [==============================] - 1s 92us/step - loss: 0.1146 - acc: 0.9551 - val_loss: 0.2004 - val_acc: 0.9196

Epoch 00177: val_acc did not improve
Epoch 178/300
5520/5520 [==============================] - 1s 95us/step - loss: 0.1154 - acc: 0.9554 - val_loss: 0.2018 - val_acc: 0.9217

Epoch 00178: val_acc did not improve
Epoch 179/300
5520/5520 [==============================] - 1s 92us/step - loss: 0.1157 - acc: 0.9540 - val_loss: 0.2303 - val_acc: 0.9094

Epoch 00179: val_acc did not improve
Epoch 180/300
5520/5520 [==============================] - 1s 102us/step - loss: 0.1145 - acc: 0.9569 - val_loss: 0.2142 - val_acc: 0.

5520/5520 [==============================] - 1s 105us/step - loss: 0.1119 - acc: 0.9592 - val_loss: 0.2015 - val_acc: 0.9261

Epoch 00221: val_acc did not improve
Epoch 222/300
5520/5520 [==============================] - 1s 93us/step - loss: 0.1027 - acc: 0.9620 - val_loss: 0.1958 - val_acc: 0.9275

Epoch 00222: val_acc did not improve
Epoch 223/300
5520/5520 [==============================] - 1s 97us/step - loss: 0.0963 - acc: 0.9609 - val_loss: 0.2220 - val_acc: 0.9116

Epoch 00223: val_acc did not improve
Epoch 224/300
5520/5520 [==============================] - 1s 91us/step - loss: 0.1102 - acc: 0.9636 - val_loss: 0.1949 - val_acc: 0.9261

Epoch 00224: val_acc did not improve
Epoch 225/300
5520/5520 [==============================] - 1s 99us/step - loss: 0.1046 - acc: 0.9565 - val_loss: 0.1977 - val_acc: 0.9261

Epoch 00225: val_acc did not improve
Epoch 226/300
5520/5520 [==============================] - 1s 92us/step - loss: 0.1118 - acc: 0.9571 - val_loss: 0.2007 - val_acc: 0.

5520/5520 [==============================] - 1s 101us/step - loss: 0.0916 - acc: 0.9672 - val_loss: 0.2082 - val_acc: 0.9174

Epoch 00267: val_acc did not improve
Epoch 268/300
5520/5520 [==============================] - 1s 96us/step - loss: 0.0933 - acc: 0.9638 - val_loss: 0.1965 - val_acc: 0.9225

Epoch 00268: val_acc did not improve
Epoch 269/300
5520/5520 [==============================] - 1s 105us/step - loss: 0.0754 - acc: 0.9717 - val_loss: 0.2444 - val_acc: 0.9123

Epoch 00269: val_acc did not improve
Epoch 270/300
5520/5520 [==============================] - 1s 96us/step - loss: 0.0990 - acc: 0.9629 - val_loss: 0.2098 - val_acc: 0.9174

Epoch 00270: val_acc did not improve
Epoch 271/300
5520/5520 [==============================] - 1s 101us/step - loss: 0.1013 - acc: 0.9638 - val_loss: 0.2032 - val_acc: 0.9196

Epoch 00271: val_acc did not improve
Epoch 272/300
5520/5520 [==============================] - 1s 97us/step - loss: 0.0873 - acc: 0.9670 - val_loss: 0.2037 - val_acc: 

In [16]:
sgdr = CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle')
from keras.callbacks import ModelCheckpoint
filepath="weights_ZvsFvsS_sgd_tests.best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

clf_ann.compile(optimizer=keras.optimizers.SGD(decay=1e-4, momentum=0.9),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
clf_ann.fit(X_train, y_train,
            batch_size = 256,
            epochs = 30,
            validation_data=(X_test,y_test),
            callbacks=[sgdr,checkpoint])
# Predicting the Test set results
y_pred = clf_ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(classification_report(y_test, y_pred, target_names=['Seizure', 'Tumor Area','Healthy']))

Train on 5520 samples, validate on 1380 samples
Epoch 1/30
5520/5520 [==============================] - 1s 195us/step - loss: 0.0882 - acc: 0.9667 - val_loss: 0.2128 - val_acc: 0.9167

Epoch 00001: val_acc improved from -inf to 0.91667, saving model to weights_ZvsFvsS_sgd_tests.best.h5
Epoch 2/30
5520/5520 [==============================] - 1s 134us/step - loss: 0.0843 - acc: 0.9672 - val_loss: 0.2074 - val_acc: 0.9210

Epoch 00002: val_acc improved from 0.91667 to 0.92101, saving model to weights_ZvsFvsS_sgd_tests.best.h5
Epoch 3/30
5520/5520 [==============================] - 1s 143us/step - loss: 0.0803 - acc: 0.9716 - val_loss: 0.2035 - val_acc: 0.9225

Epoch 00003: val_acc improved from 0.92101 to 0.92246, saving model to weights_ZvsFvsS_sgd_tests.best.h5
Epoch 4/30
5520/5520 [==============================] - 1s 141us/step - loss: 0.0852 - acc: 0.9703 - val_loss: 0.2014 - val_acc: 0.9239

Epoch 00004: val_acc improved from 0.92246 to 0.92391, saving model to weights_ZvsFvsS_sgd_t